In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import f1_score, accuracy_score

In [3]:
def load_predictions(dataset_name, partition, fold_number=1):
    path = './Saved_Predict_and_Proba/Folds/' + dataset_name.upper() + '/F' + str(fold_number)  + '/prob_' + partition +'.csv'
    table_pred = pd.read_csv(path)
    if dataset_name == 'zw':
        label = table_pred['norm']
        methods = table_pred.drop('norm', axis=1)
    else: 
        label = table_pred['class']
        methods = table_pred.drop('class', axis=1)
    methods = methods.drop('Unnamed: 0', axis=1)
    return label, methods

def load_dataset(dataset_name, fold_number):
    label_train, probas_train = load_predictions(dataset_name, 'train', fold_number)
    label_val, probas_val = load_predictions(dataset_name, 'val', fold_number)
    label_test, probas_test = load_predictions(dataset_name, 'test', fold_number)
    return label_train, probas_train, label_test, probas_test, label_val, probas_val

def filter_df_train_test(train_df, test_df, name, filter_first=True):
    train = train_df.filter(regex=name, axis=1)
    test = test_df.filter(regex=name, axis=1)
    return train, test

# Get dataset

In [4]:
dataset_name = 'union'
folds = 5
l1_ratios = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
stacking = LogisticRegression()

## Stacking group A - Varying Classification methods

In [5]:
algorithms_list = ['SVM', 'MLP', 'KNN', 'RF', 'EXTRA', 'CNN', 'LR', 'NB']
results_A = np.zeros((len(algorithms_list), folds))

for fold in range(folds):
    _, _, label_test, probas_test, label_val, probas_val = load_dataset(dataset_name, fold+1)
    for idx_alg, algorithm in enumerate(algorithms_list):
        X_val, X_test = filter_df_train_test(probas_val, probas_test, algorithm)
        stacking.fit(X_val, label_val)
        y_pred = stacking.predict(X_test)
        score = f1_score(label_test, y_pred, average='macro')
        results_A[idx_alg, fold] = score

In [14]:
results_A

array([[0.86865587, 0.85882384, 0.86805118, 0.87089876, 0.87102132],
       [0.85287782, 0.8526168 , 0.85853927, 0.85451392, 0.86092515],
       [0.75115907, 0.7232109 , 0.75116799, 0.74059108, 0.74797236],
       [0.85306091, 0.85714054, 0.85949417, 0.85594684, 0.84887681],
       [0.83851673, 0.8368226 , 0.84585362, 0.84472615, 0.82032906],
       [0.86778865, 0.86687413, 0.86302244, 0.86976729, 0.86389219],
       [0.87662882, 0.8758988 , 0.87998659, 0.88616935, 0.87558469],
       [0.84287598, 0.8330794 , 0.84724942, 0.84587613, 0.83558659]])

# Stacking group B - Varying Feature Representation methods

In [7]:
fe_list = ['CV', 'TFIDF', 'W2V', 'GLOVE', 'FAST']
results_B = np.zeros((len(fe_list), folds))
for fold in range(folds):
    _, _, label_test, probas_test, label_val, probas_val = load_dataset(dataset_name, fold+1)
    
    for idx_alg, fe in enumerate(fe_list):
        X_val, X_test = filter_df_train_test(probas_val, probas_test, fe)
        stacking.fit(X_val, label_val)
        y_pred = stacking.predict(X_test)
        results_B[idx_alg, fold] = f1_score(label_test, y_pred, average='macro')

In [15]:
results_B

array([[0.86457523, 0.86491328, 0.86936091, 0.86671001, 0.86355594],
       [0.86935493, 0.86594697, 0.8756521 , 0.87003102, 0.87113339],
       [0.83945699, 0.82848856, 0.83837865, 0.83767124, 0.81453153],
       [0.77393912, 0.77139373, 0.77793713, 0.78106404, 0.79097112],
       [0.82760102, 0.83297517, 0.82135025, 0.82362739, 0.83749645]])

# Stacking group C -  ALL techniques

In [9]:
results_C = np.zeros(folds)

for fold in range(folds):
    _, _, label_test, probas_test, label_val, probas_val = load_dataset(dataset_name, fold+1)
    stacking.fit(probas_val, label_val)
    y_pred = stacking.predict(probas_test)
    results_C[fold] = f1_score(label_test, y_pred, average='macro')

In [16]:
results_C

array([0.8693251 , 0.86775677, 0.8762298 , 0.86471151, 0.86073762])

# Stacking group D - Proposed Selection scheme

In [19]:
names_D = {}
names_D['zw'] = [
     'EXTRA-TFIDF|NB-W2V|NB-TF|KNN-GLOVE|MLP-W2V|CNN-W2V',
     'NB-FAST|CNN-W2V|MLP-GLOVE|SVM-TF|KNN-W2V|LR-TFIDF|EXTRA-TFIDF',
     'SVM-TF|RF-TFIDF|NB-GLOVE|KNN-FAST|NB-TFIDF|NB-W2V',
     'LR-W2V|NB-W2V|SVM-TFIDF|LR-GLOVE|LR-W2V',
     'LR-GLOVE|NB-W2V|SVM-TFIDF|EXTRA-TFIDF|MLP-FAST|LR-W2V',
                 ]


names_D['td'] = [
     'MLP-FAST|KNN-GLOVE|KNN-FAST|KNN-CV|MLP-W2V|NB-CV|CNN-TFIDF|NB-FAST|RF-CV|LR-CV|CNN-CV|KNN-W2V|CNN-GLOVE',
     'MLP-FAST|KNN-GLOVE|KNN-FAST|KNN-CV|MLP-W2V|NB-CV|CNN-TFIDF|NB-FAST|RF-CV|LR-CV|CNN-CV|KNN-W2V|CNN-GLOVE',
     'MLP-FAST|KNN-GLOVE|KNN-FAST|KNN-CV|MLP-W2V|NB-CV|CNN-TFIDF|NB-FAST|RF-CV|LR-CV|CNN-CV|KNN-W2V|CNN-GLOVE',
     'MLP-FAST|KNN-GLOVE|KNN-FAST|KNN-CV|MLP-W2V|NB-CV|CNN-TFIDF|NB-FAST|RF-CV|LR-CV|CNN-CV|KNN-W2V|CNN-GLOVE',
     'MLP-FAST|KNN-GLOVE|KNN-FAST|KNN-CV|MLP-W2V|NB-CV|CNN-TFIDF|NB-FAST|RF-CV|LR-CV|CNN-CV|KNN-W2V|CNN-GLOVE',
 ]
names_D ['union']= [
     'MLP-W2V|NB-TFIDF|RF-TFIDF|KNN-TFIDF|SVM-CV|NN-W2V|NN-FAST|CNN-W2V|SM-FAST',
     'MLP-TFIDF|NB-CV|SVM-CV|CNN-TF|KNN-TFIDF|CNN-FAST|XTRA-TFIDF',
     'KNN-TFIDF|RF-CV|SVM-W2V|LR-GLOVE|NB-TFIDF|KNN-FAST|KNN-GLOVE|SVM-CV',
     'SVM-TFIDF|KNN-TFIDF|CNN-W2V|KNN-FAST|SVM-FAST|EXTRA-CV|KNN-W2V|NB-TFIDF|SVM-GLOVE|LR-W2V|MLP-FAST',
     'CNN-W2V|RF-TFIDF|NB-W2V|NB-TFIDF|MLP-TFIDF|MLP-W2V',
]

names = names_D[dataset_name]
results_D = np.zeros(folds)

for fold in range(folds):
    _, _, label_test, probas_test, label_val, probas_val = load_dataset(dataset_name, fold+1)
    X_val, X_test = filter_df_train_test(probas_val, probas_test, names[fold])
    stacking.fit(X_val, label_val)
    y_pred = stacking.predict(X_test)
    results_D[fold] = f1_score(label_test, y_pred, average='macro')

In [21]:
results_D.mean()

0.878299920042975

## Summarizing results

In [12]:
# algorithms_list_A = [alg + ' (Group A)' for alg in algorithms_list]
# group_A_df = pd.DataFrame(results_A, columns=all_stacking_names, index=algorithms_list_A)

# fe_list_B = [fe + ' (Group B)' for fe in fe_list]
# group_B_df = pd.DataFrame(results_B, columns=all_stacking_names, index=fe_list_B)

# group_B_df = pd.DataFrame(results_B.reshape(1, 1), columns=all_stacking_names, index=['Group B'])
# group_C_df = pd.DataFrame(results_D.reshape(1, 1), columns=all_stacking_names, index=['Group C'])
# group_D_df = pd.DataFrame(results_D.reshape(1, 1), columns=all_stacking_names, index=['Group D'])

In [13]:
# from pprint import pprint
# pprint(pd.concat([group_A_df, group_B_df, group_C_df, group_D_df]).round(4).to_latex(caption='dataset '+ dataset_name.upper()))